In [1]:
from config import gapi_key
import csv
import pandas as pd
import os
import requests
import gmaps

In [2]:
# Read in data from previous analysis
path = os.path.join('..', 'WeatherPy', 'weather.csv')

weather = pd.read_csv(path)

## Create a Heat Map
Create a heat map that displays the humidity for every city from the part I of the homework.

In [3]:
gmaps.configure(api_key=gapi_key)

In [4]:
locations = weather[["City Latitude", "City Longitude"]].astype(float)
humidity = weather["Humidity (%)"]

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Find ideal places to vacation

In [6]:
# Narrow down the DataFrame to find ideal weather conditions

temp = weather['Temperature (F)']
humid = weather['Humidity (%)']
clouds = weather['Cloudiness (%)']
wind = weather['Wind Speed (mph)']

ideal = weather[(temp <= 90) & (temp >= 70) & (humid <= 50) & (clouds <= 25) & (wind <= 15)]

In [7]:
import requests
import json

### Get hotel

In [8]:

# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

# For each row in ideal df, find the first hotel that appears in Google Places search

hotels_info = []

for index, row in ideal.iterrows():
    
    h_info = {}
    
    lat = ideal.loc[index, 'City Latitude']
    lng = ideal.loc[index, 'City Longitude']
    city = ideal.loc[index, 'City'].title()
    country = ideal.loc[index, 'Country'].upper()
    coordinates = f"{lat}, {lng}"

    # set up search variables
    target_coordinates = coordinates
    target_radius = 5000
    target_type = "lodging"

    # declare the variables in the parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gapi_key,
    } 

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Run the request
    response = requests.get(base_url, params=params)
    r = response.json()
    try:
        # print(r['results'][0])
        name = r['results'][0]['name']
        h_info = {
            'name': name,
            "city": city,
            "country": country,
            'location': (lat, lng) 
        }
        hotels_info.append(h_info)
    except:       
        # print(coordinates)
        continue

# print(hotels_info)

In [9]:
hotel_locations = [hotel['location'] for hotel in hotels_info]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
pop_ups = [info_box_template.format(**hotel) for hotel in hotels_info]

marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=pop_ups)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))